In [23]:
## Load necessary libraries
%matplotlib inline
from constructIDF import *
import pandas as pd
import numpy as np
import itertools
import argparse
import matplotlib.pyplot as plt

from matplotlib import rcParams

rcParams['xtick.labelsize'] = 14
rcParams['ytick.labelsize'] = 14

### Step 1: Read file with hourly rainfall records.

The rainfall observations used in this example are from the station [COOP station id USC00360821](https://www.ncdc.noaa.gov/homr/#ncdcstnid=20016672&tab=MSHR) hourly time series (previously obtained and cleaned from the National Oceanic and Atmospheric Administration National Centers for Environmental Information accessed [here](https://www.ncei.noaa.gov/data/coop-hourly-precipitation/v2/)) 


In [8]:
"[*]"


# Specify path to hourly rainfall time series


path = "/Users/tanialopez/historicalIDF/Pittsburgh2.csv"

Read data into a dataframe and examine.
Hourly rainfall observations are organized by date and record, with flags inherited from the NCEI data. For more information on the flag meanings, visit
https://www.ncei.noaa.gov/data/coop-hourly-precipitation/v2/doc/readme.csv.txt

In [12]:
station_data = pd.read_csv(path, index_col=0)
station_data.head()

,STATION,NAME,date,val,PRCP_ATTRIBUTES,year
0,USW00014762,"PITTSBURGH ALLEGHENY CO AIRPORT, PA US",1950-01-01,0.14,",,0",1950
1,USW00014762,"PITTSBURGH ALLEGHENY CO AIRPORT, PA US",1950-01-02,0.22,",,0",1950
2,USW00014762,"PITTSBURGH ALLEGHENY CO AIRPORT, PA US",1950-01-03,0.49,",,0",1950
3,USW00014762,"PITTSBURGH ALLEGHENY CO AIRPORT, PA US",1950-01-04,0.11,",,0",1950
4,USW00014762,"PITTSBURGH ALLEGHENY CO AIRPORT, PA US",1950-01-05,0.62,",,0",1950


### Step 2: Construct Annual Maximum Series

IDF curves are constructed based on the Annual Maximum Series (AMS) or Partial Duration Series (PDS). In this tutorial, and given the functions implemented in `constructIDF` we will construct IDF based on AMS. The AMS corresponds to the maximum rainfall value accumulated during a specific duration for each year in record. We want to construct IDF curves for longer durations than one hour (e.g. 2 hours, 6 hours, etc.) and we can do so by aggregating the series above to the corresponding duration

In [16]:
# Specify durations for which IDF curves will be created
# From one hour, to 3 days.

durations = [24]

In [18]:
# Reformat station data and specify durations to compute AMS
ts = AMS(path, durations)


In [19]:
# This is the data that will be fed into the AMS methods.
ts.reformatted_frame.head()

,STATION,NAME,date,val,PRCP_ATTRIBUTES,year
0,USW00014762,"PITTSBURGH ALLEGHENY CO AIRPORT, PA US",1950-01-01,0.14,",,0",1950
1,USW00014762,"PITTSBURGH ALLEGHENY CO AIRPORT, PA US",1950-01-02,0.22,",,0",1950
2,USW00014762,"PITTSBURGH ALLEGHENY CO AIRPORT, PA US",1950-01-03,0.49,",,0",1950
3,USW00014762,"PITTSBURGH ALLEGHENY CO AIRPORT, PA US",1950-01-04,0.11,",,0",1950
4,USW00014762,"PITTSBURGH ALLEGHENY CO AIRPORT, PA US",1950-01-05,0.62,",,0",1950


Calculate AMS for each duration specified above. There are two methods implemented in `constructIDF`: *fixed maxima* and *sliding maxima* algorithms. The algorithms are provided in 

> Papalexiou, S. M., Dialynas, Y. G., & Grimaldi, S. (2016). Hershfield factor revisited: Correcting annual maximum precipitation. Journal of Hydrology, 542, 884–895. https://doi.org/10.1016/j.jhydrol.2016.09.058

These two functions extract the annual maximum precipitation (AMS) from a precipitation time series. The two approaches arise from the need to account for the fact that precipitation is systematicall recorded. For example, at a meteorological station, someone will check the tipping bucket pluviographs at some fixed local time each day which is then recorded as a "daily rainfall time series" at a particular location. This case results in "fixed" records, but "fixed" records have been shown inappropriate for estimating rainfall maxima. Because rainfall is a continuous variable, discretizing it can result in biases when estimating extreme rainfall, so it is advised to estimate annual maximum series using the sliding maxima approach.

In [26]:
ts.reformatted_frame

,STATION,NAME,date,val,PRCP_ATTRIBUTES,year
0,USW00014762,"PITTSBURGH ALLEGHENY CO AIRPORT, PA US",1950-01-01,0.14,",,0",1950
1,USW00014762,"PITTSBURGH ALLEGHENY CO AIRPORT, PA US",1950-01-02,0.22,",,0",1950
2,USW00014762,"PITTSBURGH ALLEGHENY CO AIRPORT, PA US",1950-01-03,0.49,",,0",1950
3,USW00014762,"PITTSBURGH ALLEGHENY CO AIRPORT, PA US",1950-01-04,0.11,",,0",1950
4,USW00014762,"PITTSBURGH ALLEGHENY CO AIRPORT, PA US",1950-01-05,0.62,",,0",1950
5,USW00014762,"PITTSBURGH ALLEGHENY CO AIRPORT, PA US",1950-01-06,0.86,",,0",1950
6,USW00014762,"PITTSBURGH ALLEGHENY CO AIRPORT, PA US",1950-01-07,0.01,",,0",1950
7,USW00014762,"PITTSBURGH ALLEGHENY CO AIRPORT, PA US",1950-01-08,0.02,",,0",1950
8,USW00014762,"PITTSBURGH ALLEGHENY CO AIRPORT, PA US",1950-01-09,0.01,",,0",1950
9,USW00014762,"PITTSBURGH ALLEGHENY CO AIRPORT, PA US",1950-01-10,0.43,",,0",1950


In [40]:
# Calculate AMS for each duration specified above.
# AMS can be calculated using sliding maxima. (This might take some time.)
out = pd.DataFrame(ts.reformatted_frame.groupby(pd.Grouper(key='date', freq='A')).agg(lambda x: x.max()).val)
out['year'] = out.index.year
out.dropna(how='any', inplace=True)

In [41]:
#################
#               #
#  AMS TABLE    #
#               #
#################


## Take a look at the computed AMS for each duration

out.head()

,val,year
date,,
1950-12-31,2.38,1950
1951-12-31,2.32,1951
1952-12-31,2.25,1952
1999-12-31,2.79,1999
2000-12-31,4.25,2000


### Step 3: Fit Generalized Extreme Value and obtain rainfall depths

The next step is to fit a generalized extreme value distribution to each duration's AMS. Once the parameters (location, scale and shape) are estimated, these are used to retrieve the return levels (in this case, rainfall depth) for different quantiles feed into the inverse of the CDF. Usually, the quantiles are equal to the inverse of the average recurrence interval (ARI) (e.g. 1/2 = 2-year).

`constructIDF` has one method that merges all these steps, but we need to specify if we want to construct confidence intervals. The method implemented in `constructIDF` is bootstrapping, so we also need to specify the number of bootsrapped samples. Default value is 1000, and using a smaller number is not recommended.

Other specification is the confidence level, alpha, used to estimate the confidence intervals. Default is 0.9 (90% confidence interval).


In this tutorial, we will compute confidence intervals at a 90% confidence level using 1000 bootstrapped samples.

```python
ci = True
alpha = 0.9
number_bootstrap = 1000
```

In [43]:
# Specifying values
ci = True 
alpha = 0.9
number_bootstrap = 1000

In [44]:
# Feeding the data and our specifications to the method.

data = IDF(out, ci, number_bootstrap, alpha)

# Construct IDF from the data we feed above and our specifications.
# Some errors will be displayed, no worries. This will take long time because
# of the number of bootsrapped samples.
# The constructed IDF is by default for the following ARI:
# 2-, 5-, 10-, 25-, 50-, 100-, 200-year

data.construct_IDF()


/Users/tanialopez/anaconda3/lib/python3.6/site-packages/scipy/stats/_continuous_distns.py:2552: RuntimeWarning: invalid value encountered in subtract
  -pex2+logpex2-logex2)


In [107]:
#################
#               #
#  IDF TABLE    #
#               #
#################

## We can access the dataframe with confidence bounds:
data.idf
data.idf_t = data.idf.transpose()
data.idf_t

,L2-yr,L5-yr,L10-yr,L25-yr,L50-yr,L100-yr,L200-yr,2-yr,5-yr,10-yr,...,50-yr,100-yr,200-yr,U2-yr,U5-yr,U10-yr,U25-yr,U50-yr,U100-yr,U200-yr
val,1.926909,2.442314,2.715654,3.138365,3.43431,3.685332,3.916878,2.14043,2.817385,3.394627,...,5.073704,5.950965,7.002162,2.344388,3.31379,4.279315,6.179123,8.285336,11.548473,15.915501


In [66]:
data.idf_t = data.idf.transpose()

### Step 3: Generate IDF curves

We will run the next cells to create the IDF curves and plot them.
We need to pass the path where the original data was stored, a path where to store
the figure and its format.

In [74]:
"[*]"

"""
Remember to change this path
"""


savepath = "/Users/tanialopez/historical-IDF"


In [106]:
median = data.idf_t.drop([x for x in data.idf_t.columns if 'L' in x], axis=1)
median = median.drop([x for x in median.columns if 'U' in x], axis=1)


"""
NOTE: The following line will save in the same folder as this
juputer notebook the 24-hour median values that you need
to use for the future IDF curves notebook.
"""
median.transpose().to_csv('24H_Pittsburgh.csv')

In [69]:
upper_level = data.idf_t.drop([x for x in data.idf_t.columns if 'U' not in x], axis=1)
lower_level = data.idf_t.drop([x for x in data.idf_t.columns if 'L' not in x], axis=1)

In [ ]:
# This line generates a plot and saves in your folder specified above
rcParams['xtick.labelsize'] = 14
rcParams['ytick.labelsize'] = 14

fig, axs = plt.subplots(figsize=(10, 7))
median.transpose().plot(ax=axs, label='median', c='k')
plt.fill_between(np.arange(0, 7), lower_level.transpose().val.values,
                         upper_level.transpose().val.values, alpha=0.3)
plt.xticks(np.arange(0, 7), ('2', '5', '10',
                                         '25', '50', '100', '200'))
plt.legend(['Median', '95% CI'], fontsize = 18, loc=2)
plt.xlabel('Return Period (years)', fontsize=18)
plt.ylabel('Precipitation Depth (mm)', fontsize=18)
plt.title('Pittsburgh Historical 24-hour DDF curve', fontsize=22)

plt.savefig('{}/HistoricalIDF.png'.format(savepath), bbox_inches='tight')